Review and comments -> Split to sentences

Sentences -> Split to word

Word -> Processed and get sentiment

For each sentence add up the sentiment of the words

For each document add up the sentiment of the sentences

In [24]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import stopwords
import string
import pandas as pd
import mtranslate

In [25]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [26]:
df = pd.DataFrame(columns=['Content', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

#file = json.loads(open('chi.json', encoding="utf8"))
with open('chi.json', 'rb') as f:
    for line in json_lines.reader(f):

        try:
            #Check if key exists
            sentence = line['reviewBody']
            #print(line['message'])

            print("New review")
            print('---------------------------------------')

            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            
            overall_sentiment = 0
            #print(sentences)

            #Sentiment for each sentence
            for s in sentences:

                s = s.lower()
                s = s.replace('chi fitness', ' ')
                s = s.replace('chi', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)

                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)

                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized

                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)

                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                    
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                
                overall_sentiment = overall_sentiment + sentiment

            df2 = pd.DataFrame([[sentence, overall_sentiment, 'Facebook Review Chi Fitness']],columns=['Content', 'Sentiment', 'Source'])
                    
            df = pd.concat([df2,df], ignore_index=True)
                    
            # sum greater than 0 => positive sentiment
            if overall_sentiment >= 0:
                print("This document has a positive sentiment: " + str(overall_sentiment))
            elif overall_sentiment < 0:
                print("This document has a negative sentiment: " + str(overall_sentiment))
            print()
        except:
            continue

New review
---------------------------------------
Sentence:  i would like to address my concern regarding to the customer service issues to  -x fitness - starling mall management.
[('i', 'NN'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('address', 'VB'), ('my', 'PRP$'), ('concern', 'NN'), ('regarding', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('customer', 'NN'), ('service', 'NN'), ('issues', 'NNS'), ('to', 'TO'), ('-x', 'VB'), ('fitness', 'JJ'), ('-', ':'), ('starling', 'VBG'), ('mall', 'JJ'), ('management', 'NN'), ('.', '.')]
[('i', 'NN'), ('would', 'MD'), ('like', 'VB'), ('to', 'TO'), ('address', 'VB'), ('my', 'PRP$'), ('concern', 'NN'), ('regarding', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('customer', 'NN'), ('service', 'NN'), ('issues', 'NNS'), ('to', 'TO'), ('-x', 'VB'), ('fitness', 'JJ'), ('-', ':'), ('starling', 'VBG'), ('mall', 'JJ'), ('management', 'NN'), ('.', '.')]
Lemmatized word:  i
i  |  Synset('iodine.n.01')  |  <iodine.n.01: PosScore=0.0 NegScore=0.0>  |  a nonmetallic 

In [27]:
df

,Content,Sentiment,Source
0,Good place to workout . But Im tried of pickin...,0.125,Facebook Review Chi Fitness
1,"I really like this place, I go to Sunway Velo...",1.625,Facebook Review Chi Fitness
2,Was absolutely bad! When I had the agreement w...,-2.625,Facebook Review Chi Fitness
3,"Well for me, I go to Damen though it's a much ...",2.750,Facebook Review Chi Fitness
4,Hi .. I wish to feedback Chi Fitness at Queens...,-0.375,Facebook Review Chi Fitness
5,"Hi Chi Fitness, I wish to lodge a complaint on...",-1.250,Facebook Review Chi Fitness
6,Chi-Fitness IKON is one of the best fitness ce...,4.750,Facebook Review Chi Fitness
7,please becareful of leonardo from queensbay ma...,0.500,Facebook Review Chi Fitness
8,"Hi, today i went to Atria mall CHI fitness for...",-3.125,Facebook Review Chi Fitness
9,Nowdays management were duin unstoppable nons...,-0.500,Facebook Review Chi Fitness


In [28]:
df3 = pd.DataFrame(columns=['Content', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()
weird_sentiment_words = ['la', 'please']

file = json.load(open('chifitness6months.json', encoding="utf8"))
for line in file['data']:
    
    try:
        #Check if key exists
        line['comments']
        #print(line['message'])
        
        for comments in line['comments']['data']:
            print("New comment")
            print('---------------------------------------')
            sentence = comments['message']
            
            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)
            
            if len(sentences) == 0:
                continue
                
            overall_sentiment = 0
            
            #Sentiment for each sentence
            for s in sentences:
                    
                s = s.lower()
                s = s.replace('chi fitness', ' ')
                s = s.replace('chi', ' ')
                s = re.sub(r"\byin\b", "resultado", s)
                s = re.sub(r"\byang\b", "resultado", s)
                
                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)
                
                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                    
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                
                overall_sentiment = overall_sentiment + sentiment

            df2 = pd.DataFrame([[sentence, overall_sentiment, 'Facebook Comment Chi Fitness']],columns=['Content', 'Sentiment', 'Source'])
                    
            df3 = pd.concat([df2,df3], ignore_index=True)
                    
            # sum greater than 0 => positive sentiment
            if overall_sentiment >= 0:
                print("This document has a positive sentiment: " + str(overall_sentiment))
            elif overall_sentiment < 0:
                print("This document has a negative sentiment: " + str(overall_sentiment))
        print()
    except:
        continue

New comment
---------------------------------------
Sentence:  my beloved, all of this.
[('my', 'PRP$'), ('beloved', 'VBN'), (',', ','), ('all', 'DT'), ('of', 'IN'), ('this', 'DT'), ('.', '.')]
Lemmatized word:  my
Synset not found
Lemmatized word:  beloved
Synset not found
Lemmatized word:  ,
Synset not found
Lemmatized word:  all
all  |  Synset('all.a.01')  |  <all.a.01: PosScore=0.0 NegScore=0.0>  |  quantifier; used with either mass or count nouns to indicate the whole number or amount of or every one of a class
Lemmatized word:  of
Synset not found
Lemmatized word:  this
Synset not found
Lemmatized word:  .
Synset not found
Sentence:  i just like it.
[('i', 'NN'), ('just', 'RB'), ('like', 'IN'), ('it', 'PRP'), ('.', '.')]
Lemmatized word:  i
i  |  Synset('iodine.n.01')  |  <iodine.n.01: PosScore=0.0 NegScore=0.0>  |  a nonmetallic element belonging to the halogens; used especially in medicine and photography and in dyes; occurs naturally only in combination in small quantities (as

In [29]:
final_df = pd.concat([df,df3], ignore_index=True)
final_df = final_df[final_df['Sentiment'] != 0]
final_df

,Content,Sentiment,Source
0,Good place to workout . But Im tried of pickin...,0.125,Facebook Review Chi Fitness
1,"I really like this place, I go to Sunway Velo...",1.625,Facebook Review Chi Fitness
2,Was absolutely bad! When I had the agreement w...,-2.625,Facebook Review Chi Fitness
3,"Well for me, I go to Damen though it's a much ...",2.750,Facebook Review Chi Fitness
4,Hi .. I wish to feedback Chi Fitness at Queens...,-0.375,Facebook Review Chi Fitness
5,"Hi Chi Fitness, I wish to lodge a complaint on...",-1.250,Facebook Review Chi Fitness
6,Chi-Fitness IKON is one of the best fitness ce...,4.750,Facebook Review Chi Fitness
7,please becareful of leonardo from queensbay ma...,0.500,Facebook Review Chi Fitness
8,"Hi, today i went to Atria mall CHI fitness for...",-3.125,Facebook Review Chi Fitness
9,Nowdays management were duin unstoppable nons...,-0.500,Facebook Review Chi Fitness


In [30]:
len(final_df[final_df['Sentiment'] < 0])

42

In [31]:
len(final_df[final_df['Sentiment'] > 0])

122

In [32]:
final_df.to_json('chi_fitness_sentiment.json', orient='records')